In [1]:
# MODEL = "llama3.2:latest"
MODEL = "mistral:latest"
CHROMA_PATH = "chroma"
DATA_PATH = "mydocs"


In [12]:
import os
from dotenv import load_dotenv
load_dotenv()

test = os.getenv("TEST")
print(test)


Hello World


In [18]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model= MODEL)

model.invoke("What is the capital of India?")

" The capital of India is New Delhi. It functions as a city and a union territory in addition to being the administrative center of the country. New Delhi was officially declared the capital of India following the country's independence from British rule in 1947. Prior to that, it had served as the capital of British India since 1931."

In [4]:
from langchain_community.document_loaders import PyPDFLoader

def load_documents():
    print("Loading documents")
    loader = PyPDFLoader("mydocs/MonopolyRules.pdf")
    documents = loader.load()
    return documents

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [6]:
chain = prompt | model
chain.input_schema.schema()
# print(chain.invoke({"context": documents, "question": "How much money do i start with?"}))

C:\Users\ipriv\AppData\Local\Temp\ipykernel_24460\4251750727.py:2: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [7]:
chain = prompt | model
chain.input_schema.schema()
# print(chain.invoke({"context": documents, "question": "How do i build a hotel?"}))

C:\Users\ipriv\AppData\Local\Temp\ipykernel_24460\3722536129.py:2: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_text(documents: list[Document]):
    print("Splitting text")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [9]:
from langchain_community.vectorstores import Chroma
import os
import shutil

def save_to_chroma(chunks: list[Document]):
    print("Saving to Chroma")
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OllamaEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [10]:
def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)

In [11]:
generate_data_store()

Loading documents
Splitting text
Split 8 documents into 79 chunks.
don't want to buy this property, move to the space anyway 
and put the property up for auction. 
NO, there are no more properties in the bank - Advance to the 
NOCT property on which you will owe another player money. 
A few minor details:
{'producer': 'Adobe Acrobat 7.0 Paper Capture Plug-in', 'creator': 'Adobe Acrobat 7.0', 'creationdate': '2007-05-03T12:38:10-04:00', 'moddate': '2007-05-03T12:52:41-04:00', 'source': 'mydocs/MonopolyRules.pdf', 'total_pages': 8, 'page': 1, 'page_label': '2', 'start_index': 630}
Saving to Chroma


C:\Users\ipriv\AppData\Local\Temp\ipykernel_24460\728343137.py:13: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  chunks, OllamaEmbeddings(), persist_directory=CHROMA_PATH


Saved 79 chunks to chroma.


C:\Users\ipriv\AppData\Local\Temp\ipykernel_24460\728343137.py:15: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [ ]:
import argparse

# Create CLI.
parser = argparse.ArgumentParser()
parser.add_argument("query_text", type=str, help="The query text.")
args = parser.parse_args()
query_text = args.query_text

# Prepare the DB.
embedding_function = OllamaEmbeddings()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

results

usage: ipykernel_launcher.py [-h] query_text
ipykernel_launcher.py: error: the following arguments are required: query_text


SystemExit: 2

c:\Users\ipriv\Documents\Personal Projects\RAG Application\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
